# TrenD 1주차 - 회귀 분석 / 가설 검정

- Stasmodels : 통계 및 회귀 분석용 라이브러리
- mtcars : 자동차 성능 데이터(마력, 연비, 기통 수, 무게 등의 정보 포함)

In [2]:
import statsmodels.api as sm

# mtcars 데이터 불러오기
df = sm.datasets.get_rdataset("mtcars", "datasets").data
df.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
rownames,,,,,,,,,,,
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


- mpg : 연비(Miles per gallon)
- cyl : 실린더 수
- disp : 배기량
- hp : 마력(Horsepower)
- drat : 후륜 기어비
- wt : 차량 무게
- qsec : 1/4마일 주파 시간
- vs : 엔진 형태(0 = 자동, 1 = 수동)
- am : 변속기 종류(0 = 자동, 1 = 수동)
- gear : 전진 기어 수
- carb : 카뷰레터 수

In [3]:
import statsmodels.api as sm

# 독립변수(X)와 종속변수(y) 설정
X = df["hp"]  # 마력
y = df["mpg"]  # 연비

# 상수항 추가
X = sm.add_constant(X)

# 회귀 모델 적합
model = sm.OLS(y, X).fit()

# 결과 출력
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.602
Model:                            OLS   Adj. R-squared:                  0.589
Method:                 Least Squares   F-statistic:                     45.46
Date:                Thu, 10 Apr 2025   Prob (F-statistic):           1.79e-07
Time:                        17:08:59   Log-Likelihood:                -87.619
No. Observations:                  32   AIC:                             179.2
Df Residuals:                      30   BIC:                             182.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         30.0989      1.634     18.421      0.000      26.762      33.436
hp            -0.0682      0.010     -6.742      0.000      -0.089      -0.048
==============================================================================
Omnibus:                        3.692   Durbin-Watson:                   1.134
Prob(Omnibus):                  0.158   Jarque-Bera (JB):                2.984
Skew:                           0.747   Prob(JB):                        0.225
Kurtosis:                       2.935   Cond. No.                         386.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [4]:
# 회귀계수 가져오기
intercept, coef_hp = model.params
# 회귀식 출력
print(f"회귀식: mpg = {intercept:.3f} + ({coef_hp:.3f} * hp)")

회귀식: mpg = 30.099 + (-0.068 * hp)


# 1.회귀계수에 대한 검정

### 1.1 beta_1 : 절편값

In [12]:
from scipy.stats import t

# 가설에서 설정한 β₀ 값 (대립가설이 주장하는 값)
beta0_1 = 10

# 절편(Intercept) 계수 및 표준 오차 가져오기
beta_intercept = model.params["const"]
std_err_intercept = model.bse["const"]


# t-검정 통계량 계산
t_stat_intercept = (beta_intercept - beta0_1) / std_err_intercept

# 자유도 계산 (회귀모형의 자유도 = 샘플 수 - 독립 변수 개수 - 1)
df = model.df_resid

# p-value 계산 (양측 검정)
p_value_intercept = 2 * (1 - t.cdf(abs(t_stat_intercept), df))

# 유의수준 설정
alpha = 0.05

# 검정 결과 출력
print("\n[절편(β₀) 가설검정]")
print(f"질문: 절편(β₀)이 {beta0_1}인가?")
print(f"귀무가설(H₀): β₀ ≠ {beta0_1} (절편이 {beta0_1}이 아니다)")
print(f"대립가설(H₁): β₀ = {beta0_1} (절편이 {beta0_1}이다)\n")

print(f"▶ 절편(β̂₀) 추정값: {beta_intercept:.5f}")
print(f"▶ 검정할 절편 값 (β₀): {beta0_1}")
print(f"▶ 계산된 t-값: {t_stat_intercept:.5f}")
print(f"▶ p-value: {p_value_intercept:.5f}")

if p_value_intercept < alpha:
    print(f"\np-value = {p_value_intercept:.5f} < {alpha}, 따라서 귀무가설을 기각합니다.")
    print(f"즉, 절편(β₀)은 {beta0_1}이 아닙니다.")
else:
    print(f"\np-value = {p_value_intercept:.5f} ≥ {alpha}, 따라서 귀무가설을 기각하지 않습니다.")
    print(f"즉, 절편(β₀)은 {beta0_1}과 유의하게 다르다고 볼 수 없습니다.")



[절편(β₀) 가설검정]
질문: 절편(β₀)이 10인가?
귀무가설(H₀): β₀ ≠ 10 (절편이 10이 아니다)
대립가설(H₁): β₀ = 10 (절편이 10이다)

▶ 절편(β̂₀) 추정값: 30.09886
▶ 검정할 절편 값 (β₀): 10
▶ 계산된 t-값: 12.30100
▶ p-value: 0.00000

p-value = 0.00000 < 0.05, 따라서 귀무가설을 기각합니다.
즉, 절편(β₀)은 10이 아닙니다.


### 1.2 beta_0 : 계수값

coef 열에서 hp의 계수(β) 값을 확인


P 값이 0.05 미만이면 귀무가설 기각 → hp가 mpg에 유의한 영향을 미친다고 해석



P 값이 0.05 이상이면 귀무가설 채택 → hp가 mpg에 유의한 영향을 미치지 않는다고 해석

In [13]:
from scipy.stats import t

# 검정할 β1 값 설정 (예: -0.05)
beta1_0 = -0.05

# 회귀 계수 및 표준 오차 가져오기
beta_hp = model.params["hp"]  # hp의 회귀 계수 (추정값)
std_err_hp = model.bse["hp"]  # hp의 표준 오차

# t-검정 통계량 계산
t_stat_hp = (beta_hp - beta1_0) / std_err_hp

# 자유도 계산 (회귀모형의 자유도 = 샘플 수 - 독립 변수 개수 - 1)
df = model.df_resid

# p-value 계산 (양측 검정)
p_value_hp = 2 * (1 - t.cdf(abs(t_stat_hp), df))

# 유의수준 설정
alpha = 0.05

# 검정 결과 출력
print("\n[회귀 계수(β) 가설검정]")
print(f"질문: 마력(hp)의 회귀 계수(β)가 {beta1_0}과 유의한 차이가 있는가?")
print(f"귀무가설(H₀): β = {beta1_0} (hp의 회귀 계수가 {beta1_0}과 같다)")
print(f"대립가설(H₁): β ≠ {beta1_0} (hp의 회귀 계수가 {beta1_0}과 다르다)\n")

print(f"▶ hp의 회귀 계수 추정값 (β̂): {beta_hp:.5f}")
print(f"▶ 검정할 β 값 (β₀): {beta1_0}")
print(f"▶ 계산된 t-값: {t_stat_hp:.5f}")
print(f"▶ p-value: {p_value_hp:.5f}")

if p_value_hp < alpha:
    print(f"\np-value = {p_value_hp:.5f} < {alpha}, 따라서 귀무가설을 기각합니다.")
    print(f"즉, hp의 회귀 계수(β)는 {beta1_0}과 유의하게 다릅니다.")
else:
    print(f"\np-value = {p_value_hp:.5f} ≥ {alpha}, 따라서 귀무가설을 기각하지 않습니다.")
    print(f"즉, hp의 회귀 계수(β)는 {beta1_0}과 유의한 차이가 있다고 볼 수 없습니다.")



[회귀 계수(β) 가설검정]
질문: 마력(hp)의 회귀 계수(β)가 -0.05과 유의한 차이가 있는가?
귀무가설(H₀): β = -0.05 (hp의 회귀 계수가 -0.05과 같다)
대립가설(H₁): β ≠ -0.05 (hp의 회귀 계수가 -0.05과 다르다)

▶ hp의 회귀 계수 추정값 (β̂): -0.06823
▶ 검정할 β 값 (β₀): -0.05
▶ 계산된 t-값: -1.80134
▶ p-value: 0.08171

p-value = 0.08171 ≥ 0.05, 따라서 귀무가설을 기각하지 않습니다.
즉, hp의 회귀 계수(β)는 -0.05과 유의한 차이가 있다고 볼 수 없습니다.


# 2.두 모집단의 평균 차이가 유의한지 검정

In [14]:
df = sm.datasets.get_rdataset("mtcars", "datasets").data
import pandas as pd
import scipy.stats as stats



# 4기통(cyl == 4)과 6기통(cyl == 6) 자동차 그룹 생성
group1 = df[df["cyl"] == 4]["mpg"]  # 4기통 자동차의 연비
group2 = df[df["cyl"] == 6]["mpg"]  # 6기통 자동차의 연비

# 독립 t-검정 수행
t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)  # 등분산 가정 X (Welch’s t-test)
alpha = 0.05  # 유의수준 5%

# 평균값 계산
mean1, mean2 = group1.mean(), group2.mean()

# 검정 결과 해석
print("\n[두 집단 평균 비교 (독립 t-검정)]")
print("질문: 4기통 자동차와 6기통 자동차의 평균 연비는 유의한 차이가 있는가?")
print(f"귀무가설(H₀): μ₁ = μ₂ (4기통과 6기통 자동차의 평균 연비가 같다)")
print(f"대립가설(H₁): μ₁ ≠ μ₂ (4기통과 6기통 자동차의 평균 연비가 다르다)\n")

print(f"▶ 4기통 자동차 평균 연비 (μ₁): {mean1:.5f}")
print(f"▶ 6기통 자동차 평균 연비 (μ₂): {mean2:.5f}")
print(f"▶ 계산된 t-값: {t_stat:.5f}")
print(f"▶ p-value: {p_value:.5f}")

# 가설 검정 결과 출력
if p_value < alpha:
    print(f"\np-value = {p_value:.5f} < {alpha}, 따라서 귀무가설을 기각합니다.")
    if mean1 > mean2:
        print(f"μ₁ = {mean1:.5f} > μ₂ = {mean2:.5f}, 따라서 4기통 자동차가 평균적으로 연비가 더 높습니다.")
    else:
        print(f"μ₁ = {mean1:.5f} < μ₂ = {mean2:.5f}, 따라서 6기통 자동차가 평균적으로 연비가 더 높습니다.")
else:
    print(f"\np-value = {p_value:.5f} ≥ {alpha}, 따라서 귀무가설을 기각하지 않습니다.")
    print(f"즉, 4기통과 6기통 자동차의 평균 연비 차이가 유의하지 않다고 볼 수 있습니다.")


[두 집단 평균 비교 (독립 t-검정)]
질문: 4기통 자동차와 6기통 자동차의 평균 연비는 유의한 차이가 있는가?
귀무가설(H₀): μ₁ = μ₂ (4기통과 6기통 자동차의 평균 연비가 같다)
대립가설(H₁): μ₁ ≠ μ₂ (4기통과 6기통 자동차의 평균 연비가 다르다)

▶ 4기통 자동차 평균 연비 (μ₁): 26.66364
▶ 6기통 자동차 평균 연비 (μ₂): 19.74286
▶ 계산된 t-값: 4.71906
▶ p-value: 0.00040

p-value = 0.00040 < 0.05, 따라서 귀무가설을 기각합니다.
μ₁ = 26.66364 > μ₂ = 19.74286, 따라서 4기통 자동차가 평균적으로 연비가 더 높습니다.


# 3.단일 집단 평균 검정

In [15]:
import pandas as pd
from scipy import stats

df = sm.datasets.get_rdataset("mtcars", "datasets").data
# 가설 설정
alpha = 0.05  # 유의수준 5%
pop_mean = 25  # 비교 대상 평균 (예: 25 mpg)

# 단일집단 t-검정 수행 (H₀: μ = 25 vs H₁: μ > 25)
t_stat, p_value = stats.ttest_1samp(df['mpg'], popmean=pop_mean)

# 단측검정이므로 p-value를 2로 나눔
one_sided_p = p_value / 2

# 평균 계산
sample_mean = df['mpg'].mean()

# 결과 출력
print("\n[단일집단 평균 검정]")
print(f"질문: 자동차 연비(mpg)의 평균이 25보다 큰가?")
print(f"귀무가설(H₀): μ = {pop_mean} (자동차 연비의 평균이 {pop_mean}이다.)")
print(f"대립가설(H₁): μ > {pop_mean} (자동차 연비의 평균이 {pop_mean}보다 크다.)\n")

print(f"▶ 표본의 평균 연비 (μ̂): {sample_mean:.3f}")
print(f"▶ 계산된 t-값: {t_stat:.3f}")
print(f"▶ 단측검정 p-value: {one_sided_p:.5f}")

# 가설 검정 결과 해석
if one_sided_p < alpha:
    print(f"\np-value = {one_sided_p:.5f} < {alpha}, 따라서 귀무가설을 기각합니다.")
    print(f"자동차 연비의 평균이 {pop_mean}보다 크다고 볼 수 있습니다.")
else:
    print(f"\np-value = {one_sided_p:.5f} ≥ {alpha}, 따라서 귀무가설을 기각하지 않습니다.")
    print(f"즉, 자동차 연비의 평균이 {pop_mean}보다 크다고 볼 충분한 증거가 없습니다.")



[단일집단 평균 검정]
질문: 자동차 연비(mpg)의 평균이 25보다 큰가?
귀무가설(H₀): μ = 25 (자동차 연비의 평균이 25이다.)
대립가설(H₁): μ > 25 (자동차 연비의 평균이 25보다 크다.)

▶ 표본의 평균 연비 (μ̂): 20.091
▶ 계산된 t-값: -4.608
▶ 단측검정 p-value: 0.00003

p-value = 0.00003 < 0.05, 따라서 귀무가설을 기각합니다.
자동차 연비의 평균이 25보다 크다고 볼 수 있습니다.


# 4.ANOVA 분산분석

In [16]:
import pandas as pd
from scipy.stats import f_oneway

# 가설 설정
alpha = 0.05  # 유의수준 5%

# 그룹별 연비 데이터 추출
mpg_4cyl = df[df["cyl"] == 4]["mpg"]
mpg_6cyl = df[df["cyl"] == 6]["mpg"]
mpg_8cyl = df[df["cyl"] == 8]["mpg"]

# ANOVA(일원분산분석) 수행
f_stat, p_value = f_oneway(mpg_4cyl, mpg_6cyl, mpg_8cyl)

# 각 그룹 평균 계산
mean_4cyl = mpg_4cyl.mean()
mean_6cyl = mpg_6cyl.mean()
mean_8cyl = mpg_8cyl.mean()

# 결과 출력
print("\n[ANOVA (일원분산분석)]")
print("질문: 4기통, 6기통, 8기통 자동차의 평균 연비가 유의하게 다른가?")
print(f"귀무가설(H₀): μ₁ = μ₂ = μ₃ (각 기통별 평균 연비가 같다)")
print(f"대립가설(H₁): 적어도 한 그룹의 평균이 다르다.\n")

print(f"▶ 4기통 자동차 평균 연비 (μ₁): {mean_4cyl:.3f}")
print(f"▶ 6기통 자동차 평균 연비 (μ₂): {mean_6cyl:.3f}")
print(f"▶ 8기통 자동차 평균 연비 (μ₃): {mean_8cyl:.3f}")
print(f"▶ 계산된 F-값: {f_stat:.3f}")
print(f"▶ p-value: {p_value:.5f}")

# 가설 검정 결과 해석
if p_value < alpha:
    print(f"\np-value = {p_value:.5f} < {alpha}, 따라서 귀무가설을 기각합니다.")
    print(f"즉, 적어도 한 그룹의 평균 연비가 다른 것으로 볼 수 있습니다.")
else:
    print(f"\np-value = {p_value:.5f} ≥ {alpha}, 따라서 귀무가설을 기각하지 않습니다.")
    print(f"즉, 4기통, 6기통, 8기통 자동차의 평균 연비 차이가 유의하지 않다고 볼 수 있습니다.")



[ANOVA (일원분산분석)]
질문: 4기통, 6기통, 8기통 자동차의 평균 연비가 유의하게 다른가?
귀무가설(H₀): μ₁ = μ₂ = μ₃ (각 기통별 평균 연비가 같다)
대립가설(H₁): 적어도 한 그룹의 평균이 다르다.

▶ 4기통 자동차 평균 연비 (μ₁): 26.664
▶ 6기통 자동차 평균 연비 (μ₂): 19.743
▶ 8기통 자동차 평균 연비 (μ₃): 15.100
▶ 계산된 F-값: 39.698
▶ p-value: 0.00000

p-value = 0.00000 < 0.05, 따라서 귀무가설을 기각합니다.
즉, 적어도 한 그룹의 평균 연비가 다른 것으로 볼 수 있습니다.


# 5.상관관계 분석

In [17]:
from scipy.stats import pearsonr

# 데이터가 제대로 로드되었는지 확인
print(df[["hp", "mpg"]].head())

# 귀무가설(H₀): ρ = 0 (hp와 mpg는 상관이 없다)
# 대립가설(H₁): ρ = -0.7 (hp와 mpg의 상관계수는 -0.7이다)
rho_0 = -0.7

# Pearson 상관계수 계산
corr, p_value = pearsonr(df["hp"], df["mpg"])

alpha = 0.05  # 유의수준 5%

print("\n[상관 분석 결과]")
print("질문: 마력(hp)과 연비(mpg)의 상관계수는 -0.7인가?")
print(f"귀무가설(H₀): ρ = {rho_0} (hp와 mpg의 상관계수가 {rho_0}이다)")
print(f"대립가설(H₁): ρ ≠ {rho_0} (hp와 mpg의 상관계수가 {rho_0}가 아니다)\n")

print(f"▶ 계산된 상관계수(ρ̂): {corr:.5f}")
print(f"▶ p-value: {p_value:.5f}")

if p_value < alpha:
    print(f"\np-value = {p_value:.5f} < {alpha}, 따라서 귀무가설을 기각합니다.")
    print(f"즉, hp와 mpg의 상관계수는 {rho_0}과 유의하게 다릅니다.")
else:
    print(f"\np-value = {p_value:.5f} ≥ {alpha}, 따라서 귀무가설을 기각하지 않습니다.")
    print(f"즉, hp와 mpg의 상관계수가 {rho_0}과 유의하게 다르다고 볼 수 없습니다.")


                    hp   mpg
rownames                    
Mazda RX4          110  21.0
Mazda RX4 Wag      110  21.0
Datsun 710          93  22.8
Hornet 4 Drive     110  21.4
Hornet Sportabout  175  18.7

[상관 분석 결과]
질문: 마력(hp)과 연비(mpg)의 상관계수는 -0.7인가?
귀무가설(H₀): ρ = -0.7 (hp와 mpg의 상관계수가 -0.7이다)
대립가설(H₁): ρ ≠ -0.7 (hp와 mpg의 상관계수가 -0.7가 아니다)

▶ 계산된 상관계수(ρ̂): -0.77617
▶ p-value: 0.00000

p-value = 0.00000 < 0.05, 따라서 귀무가설을 기각합니다.
즉, hp와 mpg의 상관계수는 -0.7과 유의하게 다릅니다.


# + 새 데이터셋

In [11]:
import pandas as pd
adult = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
                    header=None,
                    names=["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"])
print(adult.head())


   age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital-status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital-gain  capital-loss  hours-per-week  native-country  income  
0          2174             0              40   United-States   <=50